### Imports

In [119]:
import openai
import os

### Call API key

In [120]:
openai.api_key = os.getenv('OPENAI_API_KEY')

### Tell GPT how to generate the test

Creating a multiple choice quiz. The user needs to provide topic, number of multiple choices and total number of questions to be asked from the end user. 



In [121]:
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions}. " \
    +f"Each question should have {num_possible_answers} options. " \
    +f"Also include the correct answer for each question using the starting string 'Correct Answer:'"

    return prompt 

In [140]:
create_test_prompt("Python", 4, 4)

"Create a multiple choice quiz on the topic of Python consisting of 4. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer:'"

### OpenAI API Call
Using text-davinci-003 for normal text generation

In [123]:
response = openai.Completion.create(
    engine ='text-davinci-003',
    prompt=create_test_prompt('US history', 4,4),
    max_tokens=256,
    temperature = 0.5
)

In [124]:
print(response['choices'][0]['text'])



Q1. Who was the first president of the United States?
A. George Washington
B. Thomas Jefferson
C. John Adams
D. Abraham Lincoln
Correct Answer: A. George Washington

Q2. What year was the Declaration of Independence signed?
A. 1776
B. 1787
C. 1789
D. 1791
Correct Answer: A. 1776

Q3. What was the name of the first successful English settlement in the Americas?
A. Jamestown
B. Plymouth
C. New York
D. Boston
Correct Answer: A. Jamestown

Q4. What is the name of the current president of the United States?
A. Donald Trump
B. Barack Obama
C. George W. Bush
D. Bill Clinton
Correct Answer: A. Donald Trump


### Q/A Extraction

We now need to extract the questions and answers to present them to the students later

In [125]:
def create_student_view(test, num_questions):
    student_view = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] +=line+'\n'
        else:
            if question_number < num_questions:
                question_number +=1
                student_view[question_number] =''
    return student_view

In [126]:
create_student_view(response['choices'][0]['text'],4)

{1: '\n\nQ1. Who was the first president of the United States?\nA. George Washington\nB. Thomas Jefferson\nC. John Adams\nD. Abraham Lincoln\n',
 2: '\nQ2. What year was the Declaration of Independence signed?\nA. 1776\nB. 1787\nC. 1789\nD. 1791\n',
 3: '\nQ3. What was the name of the first successful English settlement in the Americas?\nA. Jamestown\nB. Plymouth\nC. New York\nD. Boston\n',
 4: '\nQ4. What is the name of the current president of the United States?\nA. Donald Trump\nB. Barack Obama\nC. George W. Bush\nD. Bill Clinton\n'}

In [127]:
result = create_student_view(response['choices'][0]['text'],4)

In [128]:
for key in result:
    print(result[key])



Q1. Who was the first president of the United States?
A. George Washington
B. Thomas Jefferson
C. John Adams
D. Abraham Lincoln


Q2. What year was the Declaration of Independence signed?
A. 1776
B. 1787
C. 1789
D. 1791


Q3. What was the name of the first successful English settlement in the Americas?
A. Jamestown
B. Plymouth
C. New York
D. Boston


Q4. What is the name of the current president of the United States?
A. Donald Trump
B. Barack Obama
C. George W. Bush
D. Bill Clinton



### Extract the correct answers

In [129]:
def extract_answer(test, num_questions):
    answers = {1:''}
    question_number = 1
    for line in test.split('\n'):
        if line.startswith('Correct Answer:'):
            answers[question_number] +=line+'\n'

            if question_number < num_questions:
                question_number +=1
                answers[question_number] = ''
    return answers


In [130]:
extract_answer(response['choices'][0]['text'],4)

{1: 'Correct Answer: A. George Washington\n',
 2: 'Correct Answer: A. 1776\n',
 3: 'Correct Answer: A. Jamestown\n',
 4: 'Correct Answer: A. Donald Trump\n'}

In [131]:
student_view = create_student_view(response['choices'][0]['text'],4)
answers = extract_answer(response['choices'][0]['text'],4)

### Exam simulation
Based on the extracted questions, we can now simulate the exam

In [132]:
#Take the exam 
def take(student_view):
    student_answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        student_answers[question] = answer
    return student_answers

In [133]:
student_answers = take(student_view)



Q1. Who was the first president of the United States?
A. George Washington
B. Thomas Jefferson
C. John Adams
D. Abraham Lincoln


Q2. What year was the Declaration of Independence signed?
A. 1776
B. 1787
C. 1789
D. 1791


Q3. What was the name of the first successful English settlement in the Americas?
A. Jamestown
B. Plymouth
C. New York
D. Boston


Q4. What is the name of the current president of the United States?
A. Donald Trump
B. Barack Obama
C. George W. Bush
D. Bill Clinton



In [134]:
student_answers

{1: 'a', 2: 'b', 3: 'v', 4: 'd'}

In [135]:
answers

{1: 'Correct Answer: A. George Washington\n',
 2: 'Correct Answer: A. 1776\n',
 3: 'Correct Answer: A. Jamestown\n',
 4: 'Correct Answer: A. Donald Trump\n'}

### Automatic Grading
Based on the student's answers and correct answers, we can now grade the test!

In [136]:
def grade(correct_answer_dict, student_answers):
    correct_answers = 0
    for question, answer in student_answers.items():
        if answer.upper == correct_answer_dict[question][16]:
            correct_answers += 1
    grade = 100*correct_answers / len(answers)

    if grade <60:
        passed = "Failed"
    else:
        passed = "Pass"
    
    return f"{correct_answers} / {len(answers)} correct! You got {grade} grade, You {passed}"

In [137]:
grade(answers, student_answers)

'0 / 4 correct! You got 0.0 grade, You Failed'